In [10]:
import os
from pathlib import Path
from glob import glob
import pandas as pd

!pip install xlrd

     |████████████████████████████████| 96 kB 3.8 MB/s eta 0:00:011
You should consider upgrading via the '/Users/g0bel1n/PycharmProjects/pythonProject/basic_env/bin/python -m pip install --upgrade pip' command.


In [11]:
current_path = Path(os.getcwd())
root = current_path.parent.absolute()
data_folder = str(root) + '/raw data/'

In [12]:
types = ["xlsx", "csv", "xls"]

os.chdir(data_folder)

tanguy = {}

print("-- XLSX FILES --")
for file_path in glob("*.xlsx"):
    file_name = file_path.split(".")[0]
    
    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
    
print("\n-- XLS FILES --") 
for file_path in glob("*.xls"):
    file_name = file_path.split(".")[0]
    
    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
print("\n-- CSV FILES --") 
for file_path in glob("*.csv"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path)
    tanguy[file_name] = file_df



-- XLSX FILES --
Quality Minus Junk Factors Daily sheets:
-->QMJ Factors
-->HML Devil

Volatility_data_treated sheets:
-->Sheet1

Value and Momentum Everywhere Factors Monthly sheets:
-->VME Factors

bloom_data_2 sheets:
-->CL1
-->CL2
-->CL3
-->Euribor3
-->Bund10
-->FUTURES MATURITIES

Betting Against Beta Equity Factors Daily sheets:
-->BAB Factors
-->HML Devil


-- XLS FILES --
sentiment sheets:
-->SENTIMENT


-- CSV FILES --
F-F_ST_Reversal_Factor_daily


In [32]:
for key in tanguy.keys():
    if type(tanguy[key].index)!=pd.DatetimeIndex:
        date_col = [col for col in ['Date','DATE','Unnamed: 0'] if col in tanguy[key].columns]
        print(date_col)
        assert len(date_col)==1, "Date column is not unique"+ str( key)
        tanguy[key] = tanguy[key].set_index(date_col)
    tanguy[key].index.names = ["Date"]

[]


AssertionError: Date column is not uniqueHML Devil

In [33]:
tanguy["HML Devil"]

,Unnamed: 1,Aggregate Equity Portfolios,Unnamed: 3,Unnamed: 4
Date,,,,
DATE,HMLD USA,HMLD Global,HMLD Global Ex USA,HMLD Europe
1926-07-01 00:00:00,-0.002133,NaN,NaN,NaN
1926-07-02 00:00:00,0.002451,NaN,NaN,NaN
1926-07-06 00:00:00,-0.006113,NaN,NaN,NaN
1926-07-07 00:00:00,-0.003518,NaN,NaN,NaN
...,...,...,...,...
44494,-0.004083,-0.000159,0.005609,0.008334
44495,-0.003723,-0.002741,-0.001311,-0.001843
44496,-0.006364,-0.003129,0.001569,0.00168


In [25]:
tanguy['BAB factors']

,Date,BAB U.S. Equities,BAB International Equities,BAB U.S. Treasury Bonds,BAB U.S. Credit Indices,BAB U.S. Credit Indices (hedged),BAB U.S. Corporate Bonds,BAB Equity Indices,BAB Country Bonds,BAB Foreign Exchange,BAB Commodities,BAB All Assets
0,192601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,192602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,192603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,192604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1030,201111,0.005024,-0.003511,-0.002533,-0.000115,0.002302,-0.005171,-0.010160,0.001127,0.015445,-0.017467,-0.000487
1031,201112,0.018002,-0.014703,-0.004352,-0.006718,-0.004888,0.016589,0.017178,-0.001785,0.018707,-0.050767,-0.006365
1032,201201,0.005713,-0.038310,0.006599,0.006425,0.007158,0.010098,-0.001787,0.000409,0.010984,-0.035288,-0.017752
1033,201202,0.029880,0.008182,-0.005198,0.002687,0.000916,-0.007465,0.021729,0.003186,-0.043234,0.014729,0.005453


In [27]:
tanguy["CL1"] = tanguy["CL1"].set_index("Unnamed: 0")
tanguy["CL1"].index.names = ["Date"]
tanguy["CL1"]

,PX_OPEN,PX_HIGH,PX_LOW,PX_LAST,PX_VOLUME
Date,,,,,
2000-01-04,25.20,25.69,24.71,25.55,76965.0
2000-01-05,25.50,25.61,24.86,24.91,74421.0
2000-01-06,24.80,25.34,24.51,24.78,87705.0
2000-01-07,24.65,25.00,24.15,24.22,93857.0
2000-01-10,24.15,24.75,24.02,24.67,71923.0
...,...,...,...,...,...
2021-05-18,66.33,67.01,64.11,65.49,167348.0
2021-05-19,65.28,65.35,61.95,63.36,101902.0
2021-05-20,63.39,63.96,61.89,62.05,29619.0


In [29]:
tanguy["Euribor3"] = tanguy["Euribor3"].set_index("Unnamed: 0")
tanguy["Euribor3"].index.names = ["Date"]
tanguy["Euribor3"]

KeyError: "None of ['Unnamed: 0'] are in the columns"

In [19]:
tanguy["Bund10"] = tanguy["Bund10"].set_index("Unnamed: 0")
tanguy["Bund10"].index.names = ["Date"]
tanguy["Bund10"]

,PX_LAST
Date,
2000-01-03,5.478
2000-01-04,5.492
2000-01-05,5.516
2000-01-06,5.545
2000-01-07,5.443
...,...
2021-05-18,-0.103
2021-05-19,-0.110
2021-05-20,-0.109


In [20]:
tanguy["FUTURES MATURITIES"]

,UX1 Index date,UX1 Index futures,UX1 Index maturity date,UX2 Index date,UX2 Index futures,UX2 Index maturity date,UX3 Index date,UX3 Index futures,UX3 Index maturity date,FVS1 Index date,...,FVS3 Index maturity date,CL1 Comdty date,CL1 Comdty futures,CL1 Comdty maturity date,CL2 Comdty date,CL2 Comdty futures,CL2 Comdty maturity date,CL3 Comdty date,CL3 Comdty futures,CL3 Comdty maturity date
0,2004-03-26,UXK04,5/18/2004,2004-03-26,UXM04,6/15/2004,2004-03-26,UXN04,7/13/2004,2009-06-02,...,8/19/2009,2000-01-04,CLG00,1/20/2000,2000-01-04,CLH00,2/22/2000,2000-01-04,CLJ00,3/21/2000
1,2004-03-29,UXK04,5/18/2004,2004-03-29,UXM04,6/15/2004,2004-03-29,UXN04,7/13/2004,2009-06-03,...,8/19/2009,2000-01-05,CLG00,1/20/2000,2000-01-05,CLH00,2/22/2000,2000-01-05,CLJ00,3/21/2000
2,2004-03-30,UXK04,5/18/2004,2004-03-30,UXM04,6/15/2004,2004-03-30,UXN04,7/13/2004,2009-06-04,...,8/19/2009,2000-01-06,CLG00,1/20/2000,2000-01-06,CLH00,2/22/2000,2000-01-06,CLJ00,3/21/2000
3,2004-03-31,UXK04,5/18/2004,2004-03-31,UXM04,6/15/2004,2004-03-31,UXN04,7/13/2004,2009-06-05,...,8/19/2009,2000-01-07,CLG00,1/20/2000,2000-01-07,CLH00,2/22/2000,2000-01-07,CLJ00,3/21/2000
4,2004-04-01,UXK04,5/18/2004,2004-04-01,UXM04,6/15/2004,2004-04-01,UXN04,7/13/2004,2009-06-08,...,8/19/2009,2000-01-10,CLG00,1/20/2000,2000-01-10,CLH00,2/22/2000,2000-01-10,CLJ00,3/21/2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5424,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,...,NaN,2021-08-06,CLU1,#N/A Invalid Security,2021-08-06,CLV1,#N/A Invalid Security,2021-08-06,CLX1,10/20/2021
5425,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,...,NaN,2021-08-09,CLU1,#N/A Invalid Security,2021-08-09,CLV1,#N/A Invalid Security,2021-08-09,CLX1,10/20/2021
5426,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,...,NaN,2021-08-10,CLU1,#N/A Invalid Security,2021-08-10,CLV1,#N/A Invalid Security,2021-08-10,CLX1,10/20/2021
5427,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,...,NaN,2021-08-11,CLU1,#N/A Invalid Security,2021-08-11,CLV1,#N/A Invalid Security,2021-08-11,CLX1,10/20/2021


In [24]:

type(tanguy["Bund10"].index)==pd.DatetimeIndex

True